In [4]:
import numpy
import io
from itertools import permutations

In [83]:
def read_file_data_train(fname, nwordstrain, nwordstest):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    sentences = []
    tags = []
    
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    
    for i in range(nwordstrain+nwordstest):
        if i<nwordstrain:
            prev_tag = '<start>'
#             print(content[idx_line])
            while not content[idx_line].startswith('</kalimat'):
                if  not content[idx_line].startswith('<kalimat'):
                    content_part = content[idx_line].split('\t')
                    if content_part[1] in tag_count:
                        tag_count[content_part[1]] += 1
                    else:
                        tag_count[content_part[1]] = 1

                    current_word_tag = (content_part[0],content_part[1])
                    if current_word_tag in word_tag:
                        word_tag[current_word_tag] += 1
                    else:    
                        word_tag[current_word_tag] = 1

                    if is_first_word == 1:
                        current_tag_trans = ('<start>',content_part[1])
                        is_first_word = 0
                    else:
                        current_tag_trans = (prev_tag,content_part[1])

                    if current_tag_trans in tag_trans:
                        tag_trans[current_tag_trans] += 1
                    else:
                        tag_trans[current_tag_trans] = 1                    
                    prev_tag = content_part[1]   

                else:
                    tag_count['<start>'] += 1
                    is_first_word = 1
                idx_line = idx_line + 1
            idx_line = idx_line+1 
        else:
            sentence=[]
            tag = []
#             print(content[idx_line])
            while not content[idx_line].startswith('</kalimat'):
                if  not content[idx_line].startswith('<kalimat'):
                    content_part = content[idx_line].split('\t')
                    sentence.append(content_part[0])
                    tag.append(content_part[1])
                idx_line += 1
            sentences.append(sentence)
            tags.append(tag)
            idx_line += 1
    return tag_count, word_tag, tag_trans, idx_line, sentences, tags

In [86]:
def create_trans_prob_table(tag_trans,tag_count):
    trans_prob={}
    for i in tag_count.keys():
        for j in tag_count.keys():
            if (i,j) in tag_trans:
                trans_prob[(i,j)]=tag_trans[(i,j)]/tag_count[i]
    return trans_prob

In [92]:
def create_emission_prob_table(word_tag,tag_count):
    emission_prob={}
    for word_tag_entry in word_tag.keys():
        word_tag_split = word_tag[word_tag_entry].split('\t')
        current_word = word_tag_split[0]
        current_tag = word_tag_split[1]
        emission_prob[(current_word,current_tag)] = word_tag[word_tag_entry]/tag_count[current_tag]
    return emission_prob

In [84]:
tag_count, word_tag, tag_trans, n_line, test_sent, test_tag = read_file_data_train('Indonesian_Manually_Tagged_Corpus_ID.tsv',1000,20)
print("Jumlah Per Tag\n")
print(tag_count)
print("\n")
print("Tag Per Kata\n")
print(word_tag)
print("\n")
print("Transisi Tabel\n")
print(tag_trans)
print("\n")
print("Jumlah Baris\n")
print(n_line)
print("\n")
print("Kalimat Test\n")
print(test_sent)
print("\n")
print("Tag Test\n")
print(test_tag)

Jumlah Per Tag

{'<start>': 1000, 'NN': 6245, 'SC': 1124, 'VB': 2758, 'NNP': 3715, 'JJ': 1033, 'RB': 546, 'IN': 2167, 'Z': 2653, 'CD': 2276, 'CC': 776, 'PR': 491, 'PRP': 691, 'MD': 522, 'FW': 397, 'NEG': 123, 'DT': 20, 'NND': 148, 'SYM': 299, 'RP': 18, 'WH': 8, 'OD': 93, 'X': 14, 'UH': 4}


Tag Per Kata

{('Kera', 'NN'): 1, ('untuk', 'SC'): 122, ('amankan', 'VB'): 1, ('pesta olahraga', 'NN'): 1, ('Pemerintah', 'NNP'): 2, ('kota', 'NNP'): 3, ('Delhi', 'NNP'): 5, ('mengerahkan', 'VB'): 1, ('monyet', 'NN'): 12, ('mengusir', 'VB'): 4, ('monyet-monyet', 'NN'): 2, ('lain', 'JJ'): 29, ('yang', 'SC'): 513, ('berbadan', 'VB'): 2, ('lebih', 'RB'): 79, ('kecil', 'JJ'): 14, ('dari', 'IN'): 235, ('arena', 'NN'): 2, ('Pesta Olahraga', 'NNP'): 1, ('Persemakmuran', 'NNP'): 1, ('.', 'Z'): 981, ('Beberapa', 'CD'): 1, ('laporan', 'NN'): 14, ('menyebutkan', 'VB'): 10, ('setidaknya', 'RB'): 2, ('10', 'CD'): 21, ('ditempatkan', 'VB'): 2, ('di', 'IN'): 347, ('luar', 'NN'): 5, ('lomba', 'NN'): 1, ('dan', 'CC'

In [93]:
emission_prob = create_emission_prob_table(word_tag,tag_count)
emission_prob

AttributeError: 'int' object has no attribute 'split'

In [87]:
trans_prob_tab = create_trans_prob_table(tag_trans,tag_count)
trans_prob_tab

{('<start>', 'NN'): 0.313,
 ('<start>', 'SC'): 0.064,
 ('<start>', 'VB'): 0.029,
 ('<start>', 'NNP'): 0.162,
 ('<start>', 'JJ'): 0.004,
 ('<start>', 'RB'): 0.012,
 ('<start>', 'IN'): 0.173,
 ('<start>', 'Z'): 0.026,
 ('<start>', 'CD'): 0.026,
 ('<start>', 'CC'): 0.085,
 ('<start>', 'PR'): 0.016,
 ('<start>', 'PRP'): 0.072,
 ('<start>', 'MD'): 0.001,
 ('<start>', 'FW'): 0.003,
 ('<start>', 'NEG'): 0.001,
 ('<start>', 'DT'): 0.004,
 ('<start>', 'NND'): 0.005,
 ('<start>', 'WH'): 0.001,
 ('<start>', 'UH'): 0.003,
 ('NN', 'NN'): 0.32602081665332266,
 ('NN', 'SC'): 0.06693354683746998,
 ('NN', 'VB'): 0.04547638110488391,
 ('NN', 'NNP'): 0.07910328262610088,
 ('NN', 'JJ'): 0.07638110488390712,
 ('NN', 'RB'): 0.01809447558046437,
 ('NN', 'IN'): 0.08310648518815052,
 ('NN', 'Z'): 0.07830264211369095,
 ('NN', 'CD'): 0.051240992794235385,
 ('NN', 'CC'): 0.03698959167333867,
 ('NN', 'PR'): 0.0499599679743795,
 ('NN', 'PRP'): 0.03186549239391513,
 ('NN', 'MD'): 0.01489191353082466,
 ('NN', 'FW'): 